In [113]:
import pybithumb
import time
import datetime
import pandas as pd
import numpy as np

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [114]:
def dataset():
    tickers = pybithumb.get_tickers()
    new_df = pd.DataFrame()
    for ticker in tickers:
        df = pybithumb.get_ohlcv(ticker)
        df = df[df.index > '2018']
        df['ticker'] = ticker
        new_df = new_df.append(df)
    return new_df

In [115]:
df = dataset()
print(df.shape)
df.head(3)

(29139, 6)


,open,high,low,close,volume,ticker
2018-01-02,18698000.0,19080000.0,18510000.0,18772000.0,11756.222617,BTC
2018-01-03,18773000.0,20810000.0,18704000.0,19970000.0,25268.176388,BTC
2018-01-04,19958000.0,20789000.0,19865000.0,20373000.0,13532.259194,BTC


# Preprocessing

In [116]:
# 캔들추가
df['candle'] = np.where(df['close'] / df['open'] > 1.01, 1, 0)
print(df.shape)
df.head(2)

(29139, 7)


,open,high,low,close,volume,ticker,candle
2018-01-02,18698000.0,19080000.0,18510000.0,18772000.0,11756.222617,BTC,0
2018-01-03,18773000.0,20810000.0,18704000.0,19970000.0,25268.176388,BTC,1


In [117]:
# 전일 5일 이동평균선 추가
def add_ma5(df):
    tickers = pybithumb.get_tickers()
    new_df = pd.DataFrame()
    for ticker in tickers:
        ticker_df = df[df['ticker'] == ticker]
        ticker_df['ma5-1day'] = ticker_df['close'].rolling(window=5).mean().shift(1)
        new_df = new_df.append(ticker_df)
    new_df.dropna(inplace=True)
    return new_df

In [123]:
df = add_ma5(df)
print(df.shape)
df.head(2)

(28253, 8)


,open,high,low,close,volume,ticker,candle,ma5-1day
2018-01-12,19404000.0,20036000.0,17980000.0,19698000.0,13164.018754,BTC,1,22129400.0
2018-01-13,19690000.0,21200000.0,19205000.0,20662000.0,11687.757348,BTC,1,21060400.0


In [137]:
# noise 추가
df['noise'] = np.round(1-(abs(df['open']-df['close']) / (df['high']-df['low'])), 2) 
print(df.shape)
df.head(2)

(28253, 9)


,open,high,low,close,volume,ticker,candle,ma5-1day,noise
2018-01-12,19404000.0,20036000.0,17980000.0,19698000.0,13164.018754,BTC,1,22129400.0,0.86
2018-01-13,19690000.0,21200000.0,19205000.0,20662000.0,11687.757348,BTC,1,21060400.0,0.51


In [ ]:
# noise 5일전 추가
